In [2]:
import pandas as pd
import numpy as np
import random
import cv2

In [1]:
# def click_event(event, x, y, flags, param):
#     if event == cv2.EVENT_LBUTTONDOWN:
#         print(f"Coordinates: ({x}, {y})")
#
# image = cv2.imread('layout/supermarket_map.jpg')
# cv2.imshow('Image', image)
# cv2.setMouseCallback('Image', click_event)
#
# cv2.waitKey(0)
# cv2.destroyAllWindows()

#  Part 1- modeling the supermarket to the map and updating departments

In [53]:
df = pd.read_csv('data/supermarket_products.csv')

In [54]:
df.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,department,aisle
0,0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,1,2,All-Seasons Salt,104,13,pantry,spices seasonings
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,beverages,tea
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen,frozen meals
4,4,5,Green Chile Anytime Sauce,5,13,pantry,marinades meat preparation


In [55]:
df.shape

(49688, 7)

In [56]:
# pre process
df = df[df['department_id'] != 21]
df.loc[df['aisle_id'] == 24, 'department_id'] = 22
df.loc[df['aisle_id'] == 24, 'department'] = 'fresh fruits'

df.loc[df['aisle_id'] == 83, 'department_id'] = 23
df.loc[df['aisle_id'] == 83, 'department'] = 'fresh vegetables'

df.loc[df['aisle_id'] == 86, 'department_id'] = 24
df.loc[df['aisle_id'] == 86, 'department'] = 'eggs'

df.loc[df['aisle_id'] == 91, 'department_id'] = 25
df.loc[df['aisle_id'] == 91, 'department'] = 'soy'

df.loc[df['aisle_id'] == 84, 'department_id'] = 26
df.loc[df['aisle_id'] == 84, 'department'] = 'milk'

df.loc[df['aisle_id'] == 66, 'department_id'] = 27
df.loc[df['aisle_id'] == 66, 'department'] = 'asian foods'

df.loc[df['aisle_id'] == 33, 'department_id'] = 6
df.loc[df['aisle_id'] == 33, 'department'] = 'kosher foods'

df.loc[df['aisle_id'] == 30, 'department_id'] = 28
df.loc[df['aisle_id'] == 30, 'department'] = 'latino foods'

df.loc[df['aisle_id'] == 76, 'department_id'] = 29
df.loc[df['aisle_id'] == 76, 'department'] = 'indian foods'

df.loc[df['department_id'] == 16, 'department'] = 'cheese'

df = df.reset_index(drop=True)

In [57]:
department_dict = {
                    1: 'fresh fruits',
                    2: 'fresh vegetables',
                    3: 'produce',
                    4: 'meat seafood',
                    5:  'bulk',
                    6: 'other',
                    7: 'dry goods pasta',
                    8: 'pantry',
                    9: 'breakfast',
                    10: 'pets',
                    11: 'bakery',
                    12: 'deli',
                    13: 'frozen',
                    14: 'babies',
                    15: 'eggs',
                    16: 'personal care',
                    17: 'snacks',
                    18: 'household',
                    19: 'alcohol',
                    20: 'cheese',
                    21: 'milk',
                    22: 'soy',
                    23: 'canned goods',
                    24: 'beverages',
                    25: 'asian foods',
                    26: 'latino foods',
                    27: 'kosher foods',
                    28: 'indian foods',
                    29: 'none',
                }

In [58]:
coordinated_dict = {
        1: [(37, 19), (115, 280)],
        2: [(140, 19), (219, 269)],
        3: [(242, 19), (325, 258)],
        4: [(344, 19), (430, 252)],
        5: [(462, 19), (548, 253)],
        6: [(569, 19), (654, 248)],
        7: [(683, 19), (770, 258)],
        8: [(797, 19), (876, 272)],
        9: [(894, 19), (982, 276)],
        10: [(38, 327), (118, 491)],
        11: [(136, 287), (357, 424)],
        12: [(394, 265), (628, 385)],
        13: [(666, 290), (874, 429)],
        14: [(895, 323), (983, 500)],
        15: [(35, 510), (115, 701)],
        16: [(137, 530), (353, 691)],
        17: [(396, 637), (623, 739)],
        18: [(667, 525), (872, 684)],
        19: [(896, 510), (977, 692)],
        20: [(796, 709), (977, 915)],
        21: [(683, 710), (768, 911)],
        22: [(555, 750), (658, 916)],
        23: [(254, 745), (525, 892)],
        24: [(38, 710), (214, 906)],
        25: [(5, 930), (325, 989)],
        26: [(340, 930), (471, 989)],
        27: [(557, 930), (715, 989)],
        28: [(736, 930), (1010, 989)],
        29: [(362, 398), (660, 600)],
    }

In [59]:
department_coordinated_dict = {}
for i in coordinated_dict.keys():
    department_coordinated_dict[department_dict[i]] = coordinated_dict[i]
department_coordinated_dict

{'fresh fruits': [(37, 19), (115, 280)],
 'fresh vegetables': [(140, 19), (219, 269)],
 'produce': [(242, 19), (325, 258)],
 'meat seafood': [(344, 19), (430, 252)],
 'bulk': [(462, 19), (548, 253)],
 'other': [(569, 19), (654, 248)],
 'dry goods pasta': [(683, 19), (770, 258)],
 'pantry': [(797, 19), (876, 272)],
 'breakfast': [(894, 19), (982, 276)],
 'pets': [(38, 327), (118, 491)],
 'bakery': [(136, 287), (357, 424)],
 'deli': [(394, 265), (628, 385)],
 'frozen': [(666, 290), (874, 429)],
 'babies': [(895, 323), (983, 500)],
 'eggs': [(35, 510), (115, 701)],
 'personal care': [(137, 530), (353, 691)],
 'snacks': [(396, 637), (623, 739)],
 'household': [(667, 525), (872, 684)],
 'alcohol': [(896, 510), (977, 692)],
 'cheese': [(796, 709), (977, 915)],
 'milk': [(683, 710), (768, 911)],
 'soy': [(555, 750), (658, 916)],
 'canned goods': [(254, 745), (525, 892)],
 'beverages': [(38, 710), (214, 906)],
 'asian foods': [(5, 930), (325, 989)],
 'latino foods': [(340, 930), (471, 989)],
 

In [60]:
def randomize_coordinate(df, department, coordinates):
    x1, y1 = coordinates[0]
    x2, y2 = coordinates[1]

    for id in df[df['department'] == department]['product_id'].unique():
        random_x = random.randint(x1, x2)
        random_y = random.randint(y1, y2)
        df.loc[df['product_id'] == id, 'x_location'] = random_x
        df.loc[df['product_id'] == id, 'y_location'] = random_y
    return df


In [61]:
df['x_location'] = 0
df['y_location'] = 0
for index, department in department_dict.items():
    df = randomize_coordinate(df, department, department_coordinated_dict[department])


In [62]:
# import pandas as pd
# import matplotlib.pyplot as plt
#
# img = plt.imread('layout/supermarket_map.jpg')
# plt.imshow(img)
#
# # Plot red dots for each (x_location, y_location) pair
# for index, row in df.iterrows():
#     plt.scatter(row['x_location'], row['y_location'], color='red', s=3)
#     # break
#
# plt.show()

In [63]:
def closest_edge_coordinate(coordinates1, coordinates2, product_location):
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    x, y = product_location

    # Calculate the center of the square
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2

    # Calculate the half-width and half-height of the square
    half_width = abs(x2 - x1) / 2
    half_height = abs(y2 - y1) / 2

    # Calculate the distances from C to the edges of the square
    dx = min(abs(x - (center_x - half_width)), abs(x - (center_x + half_width)))
    dy = min(abs(y - (center_y - half_height)), abs(y - (center_y + half_height)))

    # Determine which edge is closest and return the corresponding coordinate
    if dx < dy:
        if x < center_x:
            return (x1, y)  # Left edge
        else:
            return (x2, y)  # Right edge
    else:
        if y < center_y:
            return (x, y1)  # Top edge
        else:
            return (x, y2)

In [64]:
df[['x_closest_location', 'y_closest_location']] = df.apply(
    lambda row: pd.Series(closest_edge_coordinate(
        department_coordinated_dict[row['department']][0],
        department_coordinated_dict[row['department']][1],
        (row['x_location'], row['y_location'])
    )), axis=1)

In [65]:
# Assuming df is your DataFrame
df['closest_valid_location'] = df.apply(lambda row: closest_edge_coordinate(department_coordinated_dict[row['department']][0],                                                       department_coordinated_dict[row['department']][1], (int(row['x_location']), int(row['y_location']))), axis=1)


In [66]:
df.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,department,aisle,x_location,y_location,x_closest_location,y_closest_location,closest_valid_location
0,0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes,544,655,544,637,"(544, 637)"
1,1,2,All-Seasons Salt,104,13,pantry,spices seasonings,864,126,876,126,"(876, 126)"
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,beverages,tea,50,813,38,813,"(38, 813)"
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen,frozen meals,733,290,733,290,"(733, 290)"
4,4,5,Green Chile Anytime Sauce,5,13,pantry,marinades meat preparation,836,149,797,149,"(797, 149)"


In [67]:
df = df[['product_name', 'department', 'aisle', 'x_location', 'y_location', 'x_closest_location', 'y_closest_location']]
df['product_inventory_super1'] = [random.randint(0,10) for i in range(df.shape[0])]
df['product_inventory_super2'] = [random.randint(0,10) for i in range(df.shape[0])]
df['product_inventory_super3'] = [random.randint(0,10) for i in range(df.shape[0])]

In [68]:
df.head()

,product_name,department,aisle,x_location,y_location,x_closest_location,y_closest_location,product_inventory_super1,product_inventory_super2,product_inventory_super3
0,Chocolate Sandwich Cookies,snacks,cookies cakes,544,655,544,637,4,9,10
1,All-Seasons Salt,pantry,spices seasonings,864,126,876,126,1,7,8
2,Robust Golden Unsweetened Oolong Tea,beverages,tea,50,813,38,813,10,0,5
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen,frozen meals,733,290,733,290,8,10,7
4,Green Chile Anytime Sauce,pantry,marinades meat preparation,836,149,797,149,1,4,0


In [69]:
df.to_csv('data/final_products_supermarket.csv')

# Part 2- Create map graph

In [70]:
import numpy as np
import pandas as pd
import pickle

In [71]:
coordinated_dict = {
        1: [(37, 19), (115, 280)],
        2: [(140, 19), (219, 269)],
        3: [(242, 19), (325, 258)],
        4: [(344, 19), (430, 252)],
        5: [(462, 19), (548, 253)],
        6: [(569, 19), (654, 248)],
        7: [(683, 19), (770, 258)],
        8: [(797, 19), (876, 272)],
        9: [(894, 19), (982, 276)],
        10: [(38, 327), (118, 491)],
        11: [(136, 287), (357, 424)],
        12: [(394, 265), (628, 385)],
        13: [(666, 290), (874, 429)],
        14: [(895, 323), (983, 500)],
        15: [(35, 510), (115, 701)],
        16: [(137, 530), (353, 691)],
        17: [(396, 637), (623, 739)],
        18: [(667, 525), (872, 684)],
        19: [(896, 510), (977, 692)],
        20: [(796, 709), (977, 915)],
        21: [(683, 710), (768, 911)],
        22: [(555, 750), (658, 916)],
        23: [(254, 745), (525, 892)],
        24: [(38, 710), (214, 906)],
        25: [(5, 930), (325, 989)],
        26: [(340, 930), (471, 989)],
        27: [(557, 930), (715, 989)],
        28: [(736, 930), (1010, 989)],
        29: [(362, 398), (660, 600)],
        30: [(0, 0), (20, 937)],
        31: [(999, 2), (1022, 948)]
    }

In [72]:
def create_coordinate_graph(coordinated_dict):
    map_coordinates = np.ones((1006, 992))
    for (x1, y1), (x2, y2) in coordinated_dict.values():
        map_coordinates[y1+1:y2, x1+1:x2] = 0

    # Create a graph
    G = nx.Graph()

    # Create nodes for each cell in the map
    rows, cols = map_coordinates.shape
    for i in range(rows):
        for j in range(cols):
            if map_coordinates[i, j] == 1:  # Only consider valid paths
                G.add_node((i, j))

    for node in G.nodes:
        x, y = node
        neighbors = [(x+dx, y+dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]
        for neighbor in neighbors:
            if neighbor in G.nodes and map_coordinates[neighbor] == 1:  # Only add valid paths
                G.add_edge(node, neighbor, weight=1)  # Assuming uniform weight for edges

    file_path = 'data/map_graph.pkl'
    with open(file_path, 'wb') as f:
        pickle.dump(G, f)

create_coordinate_graph(coordinated_dict)